In [198]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [199]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [200]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "Virny":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/Virny


# Benchmark

## Import dependencies

In [201]:
import os
import pandas as pd

from datetime import datetime, timezone
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from docs.examples.benchmark_utils import clear_directory, create_averaged_dfs_dict, populate_benchmark_report
from virny.datasets.data_loaders import CompasWithoutSensitiveAttrsDataset, ACSEmploymentDataset
from virny.user_interfaces.metrics_computation_interfaces import compute_metrics_multiple_runs
from virny.utils.custom_initializers import create_config_obj, read_model_metric_dfs

## COMPAS_Without_Sensitive_Attributes Dataset

## Initialize Input Variables

### Create a models config

In [203]:
models_config = {
    'DecisionTreeClassifier': DecisionTreeClassifier(criterion='gini',
                                                     max_depth=20,
                                                     max_features=0.6,
                                                     min_samples_split=0.1),
    'LogisticRegression': LogisticRegression(C=1,
                                             max_iter=50,
                                             penalty='l2',
                                             solver='newton-cg'),
    'RandomForestClassifier': RandomForestClassifier(max_depth=4,
                                                     max_features=0.6,
                                                     min_samples_leaf=1,
                                                     n_estimators=50),
    'XGBClassifier': XGBClassifier(learning_rate=0.1,
                                   max_depth=5,
                                   n_estimators=20),
}

### Create a Dataset class

In [204]:
dataset = CompasWithoutSensitiveAttrsDataset(dataset_path=os.path.join('virny', 'datasets', 'COMPAS.csv'))
dataset.X_data.head()

,juv_fel_count,juv_misd_count,juv_other_count,priors_count,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,c_charge_degree_F,c_charge_degree_M
0,0.0,-2.340451,1.0,-15.010999,1,0,0,0,1
1,0.0,0.000000,0.0,0.000000,1,0,0,1,0
2,0.0,0.000000,0.0,0.000000,0,0,1,1,0
3,0.0,0.000000,0.0,6.000000,1,0,0,0,1
4,0.0,0.000000,0.0,7.513697,1,0,0,1,0


### Create a config object

In [205]:
ROOT_DIR = os.path.join('docs', 'examples')
config_yaml_path = os.path.join(ROOT_DIR, 'experiment_compas_config.yaml')
config_yaml_content = """
dataset_name: COMPAS_Without_Sensitive_Attributes
test_set_fraction: 0.2
bootstrap_fraction: 0.8
n_estimators: 100
runs_seed_lst: [100, 200, 300]
#runs_seed_lst: [100, 200]
sensitive_attributes_dct: {'sex': 0, 'race': 'Caucasian', 'sex&race': None}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

In [206]:
config = create_config_obj(config_yaml_path=config_yaml_path)
SAVE_RESULTS_DIR_PATH1 = os.path.join(ROOT_DIR, 'results', 'benchmark', 'benchmark_results', config.dataset_name)
STANDARD_RESULTS_DIR_PATH1 = os.path.join(ROOT_DIR, 'results', 'benchmark', 'standard_results', config.dataset_name)

In [207]:
clear_directory(SAVE_RESULTS_DIR_PATH1)

Directory is cleared


## Subgroup Metrics Computation

In [208]:
multiple_run_metrics_dct = compute_metrics_multiple_runs(dataset, config, models_config, SAVE_RESULTS_DIR_PATH1, debug_mode=False)

Multiple runs progress:   0%|          | 0/3 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/4 [00:00<?, ?it/s]

##############################  [Model 1 / 4] Analyze DecisionTreeClassifier  ##############################
Model random_state:  101
Baseline X_train shape:  (4222, 9)
Baseline X_test shape:  (1056, 9)




2023-02-04 13:52:40 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]

...

2023-02-04 13:55:08 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-02-04 13:55:11 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics






##############################  [Model 4 / 4] Analyze XGBClassifier  ##############################
Model random_state:  304
Baseline X_train shape:  (4222, 9)
Baseline X_test shape:  (1056, 9)




2023-02-04 13:55:18 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]

2023-02-04 13:55:27 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-02-04 13:55:30 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics


In [209]:
sample_model_metrics_df = multiple_run_metrics_dct[list(models_config.keys())[0]]
sample_model_metrics_df.head(20)

,Metric,overall,sex_priv,sex_dis,race_priv,race_dis,sex&race_priv,sex&race_dis,Model_Seed,Model_Name,Run_Number
0,Mean,0.527340,0.559008,0.519987,0.592497,0.483109,0.590678,0.472225,101,DecisionTreeClassifier,Run_1
1,Std,0.071515,0.077613,0.070099,0.069249,0.073053,0.087221,0.073708,101,DecisionTreeClassifier,Run_1
2,IQR,0.087940,0.094773,0.086353,0.087878,0.087982,0.111339,0.089341,101,DecisionTreeClassifier,Run_1
3,Entropy,0.000000,0.275627,0.000000,0.000000,0.270066,0.265235,0.267109,101,DecisionTreeClassifier,Run_1
4,Jitter,0.164416,0.182694,0.160172,0.143729,0.178460,0.175473,0.176325,101,DecisionTreeClassifier,Run_1
5,Per_Sample_Accuracy,0.651705,0.661558,0.649417,0.654262,0.649968,0.640227,0.643861,101,DecisionTreeClassifier,Run_1
6,Label_Stability,0.755492,0.725126,0.762544,0.781686,0.737711,0.734545,0.742008,101,DecisionTreeClassifier,Run_1
7,TPR,0.597536,0.542857,0.606715,0.433962,0.676829,0.440000,0.689046,101,DecisionTreeClassifier,Run_1
8,TNR,0.745167,0.767442,0.738636,0.805970,0.691030,0.730159,0.659574,101,DecisionTreeClassifier,Run_1
9,PPV,0.667431,0.558824,0.687500,0.570248,0.704762,0.392857,0.709091,101,DecisionTreeClassifier,Run_1


## Create a Benchmark Report

In [210]:
models_metrics_dct = read_model_metric_dfs(SAVE_RESULTS_DIR_PATH1, model_names=models_config.keys())
standard_models_metrics_dct = read_model_metric_dfs(STANDARD_RESULTS_DIR_PATH1, model_names=models_config.keys())

avg_models_metrics_dct = create_averaged_dfs_dict(models_metrics_dct)
avg_standard_models_metrics_dct = create_averaged_dfs_dict(standard_models_metrics_dct)

In [211]:
report_df = pd.DataFrame()
report_df = populate_benchmark_report(report_df, avg_models_metrics_dct, avg_standard_models_metrics_dct,
                                      config.dataset_name, config.sensitive_attributes_dct)

In [212]:
report_df

,Dataset,Model,Subgroup,Delta%_Accuracy,Delta%_Entropy,Delta%_F1,Delta%_FNR,Delta%_FPR,Delta%_General_Ensemble_Accuracy,Delta%_IQR,Delta%_Jitter,Delta%_Label_Stability,Delta%_Mean,Delta%_PPV,Delta%_Per_Sample_Accuracy,Delta%_Positive-Rate,Delta%_Selection-Rate,Delta%_Std,Delta%_TNR,Delta%_TPR
0,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,overall,-0.047,-1.439,-0.256,0.250,-0.170,58.450,-5.999,-69.491,33.628,-13.095,-0.021,-29.019,50.830,36.781,-64.511,9.012,62.359
1,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex_priv,-0.240,-20.514,-0.946,1.865,-0.750,57.287,-7.025,-64.904,27.141,1.134,-10.031,-31.637,58.808,27.042,-66.800,20.570,52.821
2,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex_dis,-0.004,-1.471,-0.120,-0.043,-0.054,58.733,-5.756,-70.571,35.158,-15.880,1.830,-28.544,48.920,39.054,-63.788,6.590,64.054
3,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,race_priv,-0.176,-1.599,-0.593,0.103,0.454,56.451,-4.955,-71.109,27.613,3.813,-10.575,-15.199,51.663,23.376,-71.398,33.293,44.380
4,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,race_dis,0.042,-13.161,-0.106,0.380,-0.634,59.755,-6.691,-68.411,37.549,-21.209,3.712,-35.316,47.904,45.569,-58.664,-4.818,70.749
5,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex&race_priv,1.195,-12.752,1.579,-0.270,-1.680,54.740,-6.153,-64.564,23.466,14.151,-14.438,-26.305,56.845,20.702,-67.255,35.097,42.449
6,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex&race_dis,0.369,-12.994,0.171,0.046,-1.115,59.847,-6.466,-69.085,39.109,-23.505,5.536,-35.090,45.238,48.190,-56.335,-7.866,72.250
7,COMPAS_Without_Sensitive_Attributes,LogisticRegression,overall,-0.253,-0.372,-0.735,1.069,-0.381,62.740,-2.448,-88.074,40.960,-12.521,-0.290,-24.404,46.014,39.540,-70.052,13.847,57.652
8,COMPAS_Without_Sensitive_Attributes,LogisticRegression,sex_priv,-1.957,0.000,-4.330,4.353,1.014,62.284,-3.079,-87.434,36.620,2.009,-14.276,-17.998,51.030,27.125,-76.960,31.878,41.670
9,COMPAS_Without_Sensitive_Attributes,LogisticRegression,sex_dis,0.145,-0.328,-0.137,0.510,-0.767,62.870,-2.304,-88.213,41.971,-15.168,1.881,-25.375,44.408,42.441,-68.039,10.120,60.354


## Folktables [GA, 2018] Dataset

In [213]:
dataset2 = ACSEmploymentDataset(state=['GA'], year=2018, root_dir=os.path.join('virny', 'datasets'), with_nulls=False, subsample=20_000)
dataset2.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
83595,5,0,1,0,0,1,2,2,2,1,2,2,1,1,0,0
33170,5,0,1,1,2,1,2,2,2,1,2,2,1,1,2,5
90575,5,4,0,1,2,1,2,2,2,2,9,2,1,4,13,32
30079,2,4,0,1,1,1,1,2,2,1,2,0,1,1,17,56
77248,1,4,0,1,2,1,1,2,2,1,1,6,1,1,17,61


In [214]:
models_config2 = {
    'DecisionTreeClassifier': DecisionTreeClassifier(criterion='gini',
                                                     max_depth=10,
                                                     max_features=0.6,
                                                     min_samples_split=0.1),
    'LogisticRegression': LogisticRegression(C=1,
                                             max_iter=150,
                                             penalty='l2',
                                             solver='lbfgs'),
    'RandomForestClassifier': RandomForestClassifier(max_depth=10,
                                                     max_features=0.6,
                                                     min_samples_leaf=4,
                                                     n_estimators=100),
    'XGBClassifier': XGBClassifier(reg_lambda=1,
                                   learning_rate=0.1,
                                   max_depth=3,
                                   n_estimators=100),
}

In [215]:
config_yaml_path = os.path.join(ROOT_DIR, 'experiment_compas_config.yaml')
config_yaml_content = """
dataset_name: Folktables_GA_2018
test_set_fraction: 0.2
bootstrap_fraction: 0.8
n_estimators: 100
runs_seed_lst: [100, 200, 300]
#runs_seed_lst: [100, 200, 300, 400, 500, 600]
sensitive_attributes_dct: {'SEX': '1', 'RAC1P': '1', 'SEX&RAC1P': None}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

config2 = create_config_obj(config_yaml_path=config_yaml_path)
SAVE_RESULTS_DIR_PATH2 = os.path.join(ROOT_DIR, 'results', 'benchmark', 'benchmark_results', config2.dataset_name)
STANDARD_RESULTS_DIR_PATH2 = os.path.join(ROOT_DIR, 'results', 'benchmark', 'standard_results', config2.dataset_name)

In [216]:
clear_directory(SAVE_RESULTS_DIR_PATH2)

Directory is cleared


In [217]:
multiple_run_metrics_dct2 = compute_metrics_multiple_runs(dataset2, config2, models_config2, SAVE_RESULTS_DIR_PATH2, debug_mode=False)

Multiple runs progress:   0%|          | 0/3 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/4 [00:00<?, ?it/s]

##############################  [Model 1 / 4] Analyze DecisionTreeClassifier  ##############################
Model random_state:  101
Baseline X_train shape:  (16000, 16)
Baseline X_test shape:  (4000, 16)




2023-02-04 13:55:41 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]

...

2023-02-04 14:10:06 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-02-04 14:10:14 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics






##############################  [Model 4 / 4] Analyze XGBClassifier  ##############################
Model random_state:  304
Baseline X_train shape:  (16000, 16)
Baseline X_test shape:  (4000, 16)




2023-02-04 14:10:37 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]

2023-02-04 14:11:15 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-02-04 14:11:25 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics


In [218]:
sample_model_metrics_df = multiple_run_metrics_dct2[list(models_config2.keys())[0]]
sample_model_metrics_df.head(20)

,Metric,overall,SEX_priv,SEX_dis,RAC1P_priv,RAC1P_dis,SEX&RAC1P_priv,SEX&RAC1P_dis,Model_Seed,Model_Name,Run_Number
0,Mean,0.554346,0.520324,0.586419,0.547753,0.567187,0.504085,0.581497,101,DecisionTreeClassifier,Run_1
1,Std,0.057353,0.056601,0.058062,0.056487,0.059040,0.055073,0.058531,101,DecisionTreeClassifier,Run_1
2,IQR,0.069112,0.068762,0.069442,0.068703,0.069907,0.068545,0.070587,101,DecisionTreeClassifier,Run_1
3,Entropy,0.066744,0.046762,0.085582,0.058144,0.083496,0.043045,0.111252,101,DecisionTreeClassifier,Run_1
4,Jitter,0.042313,0.029759,0.054147,0.036688,0.053269,0.027803,0.071811,101,DecisionTreeClassifier,Run_1
5,Per_Sample_Accuracy,0.821237,0.855090,0.789325,0.819342,0.824930,0.861309,0.807954,101,DecisionTreeClassifier,Run_1
6,Label_Stability,0.940725,0.955126,0.927149,0.948664,0.925262,0.957599,0.901688,101,DecisionTreeClassifier,Run_1
7,TPR,0.846961,0.878178,0.812933,0.836066,0.869492,0.868976,0.841935,101,DecisionTreeClassifier,Run_1
8,TNR,0.816438,0.849549,0.788768,0.816585,0.816167,0.869144,0.814910,101,DecisionTreeClassifier,Run_1
9,PPV,0.792248,0.846782,0.736402,0.796253,0.784404,0.876900,0.783784,101,DecisionTreeClassifier,Run_1


In [219]:
models_metrics_dct2 = read_model_metric_dfs(SAVE_RESULTS_DIR_PATH2, model_names=models_config2.keys())
standard_models_metrics_dct2 = read_model_metric_dfs(STANDARD_RESULTS_DIR_PATH2, model_names=models_config2.keys())

avg_models_metrics_dct2 = create_averaged_dfs_dict(models_metrics_dct2)
avg_standard_models_metrics_dct2 = create_averaged_dfs_dict(standard_models_metrics_dct2)

## Create a Benchmark Report

In [220]:
report_df = populate_benchmark_report(report_df, avg_models_metrics_dct2, avg_standard_models_metrics_dct2,
                                      config2.dataset_name, config2.sensitive_attributes_dct)

In [221]:
filename = f'benchmark_report_{datetime.now(timezone.utc).strftime("%Y%m%d__%H%M%S")}'
report_df.to_csv(os.path.join(ROOT_DIR, 'results', 'benchmark', 'benchmark_reports', f'{filename}.csv'), index=False)

In [222]:
report_df

,Dataset,Model,Subgroup,Delta%_Accuracy,Delta%_Entropy,Delta%_F1,Delta%_FNR,Delta%_FPR,Delta%_General_Ensemble_Accuracy,Delta%_IQR,Delta%_Jitter,Delta%_Label_Stability,Delta%_Mean,Delta%_PPV,Delta%_Per_Sample_Accuracy,Delta%_Positive-Rate,Delta%_Selection-Rate,Delta%_Std,Delta%_TNR,Delta%_TPR
0,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,overall,-0.047,-1.439,-0.256,0.250,-0.170,58.450,-5.999,-69.491,33.628,-13.095,-0.021,-29.019,50.830,36.781,-64.511,9.012,62.359
1,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex_priv,-0.240,-20.514,-0.946,1.865,-0.750,57.287,-7.025,-64.904,27.141,1.134,-10.031,-31.637,58.808,27.042,-66.800,20.570,52.821
2,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex_dis,-0.004,-1.471,-0.120,-0.043,-0.054,58.733,-5.756,-70.571,35.158,-15.880,1.830,-28.544,48.920,39.054,-63.788,6.590,64.054
3,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,race_priv,-0.176,-1.599,-0.593,0.103,0.454,56.451,-4.955,-71.109,27.613,3.813,-10.575,-15.199,51.663,23.376,-71.398,33.293,44.380
4,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,race_dis,0.042,-13.161,-0.106,0.380,-0.634,59.755,-6.691,-68.411,37.549,-21.209,3.712,-35.316,47.904,45.569,-58.664,-4.818,70.749
5,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex&race_priv,1.195,-12.752,1.579,-0.270,-1.680,54.740,-6.153,-64.564,23.466,14.151,-14.438,-26.305,56.845,20.702,-67.255,35.097,42.449
6,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex&race_dis,0.369,-12.994,0.171,0.046,-1.115,59.847,-6.466,-69.085,39.109,-23.505,5.536,-35.090,45.238,48.190,-56.335,-7.866,72.250
7,COMPAS_Without_Sensitive_Attributes,LogisticRegression,overall,-0.253,-0.372,-0.735,1.069,-0.381,62.740,-2.448,-88.074,40.960,-12.521,-0.290,-24.404,46.014,39.540,-70.052,13.847,57.652
8,COMPAS_Without_Sensitive_Attributes,LogisticRegression,sex_priv,-1.957,0.000,-4.330,4.353,1.014,62.284,-3.079,-87.434,36.620,2.009,-14.276,-17.998,51.030,27.125,-76.960,31.878,41.670
9,COMPAS_Without_Sensitive_Attributes,LogisticRegression,sex_dis,0.145,-0.328,-0.137,0.510,-0.767,62.870,-2.304,-88.213,41.971,-15.168,1.881,-25.375,44.408,42.441,-68.039,10.120,60.354
